# EECS 598 Python Tutorial

Adapted from the `CS231n` Python tutorial by Justin Johnson ( http://cs231n.github.io/python-numpy-tutorial ).

## Introduction

Python is a great general-purpose programming language on its own, but with the help of a few popular libraries (numpy, scipy, matplotlib) it becomes a powerful environment for scientific computing.

We expect that many of you will have some experience with Python and numpy; for the rest of you, this section will serve as a quick crash course both on the Python programming language and on the use of Python for scientific computing.

Some of you may have previous knowledge in Matlab, in which case we also recommend the numpy for Matlab users page (https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html).

In this tutorial, we will cover:

* Basic Python: Basic data types (Containers, Lists, Dictionaries, Sets, Tuples), Functions, Classes
* Numpy: Arrays, Array indexing, Datatypes, Array math, Broadcasting
* Matplotlib: Plotting, Subplots, Images
* PyTorch basics: Tensors, Operations, Linear Regression example

### Installation instructions

You will need the following python packages for this tutorial: jupyter, numpy, matplotlib, pytorch

We recommend installing Anaconda/Miniconda for package management. This makes installation of the packages and their dependencies easy.

* Installing Anaconda: https://www.anaconda.com/download
* Anaconda cheatsheet: https://conda.io/docs/_downloads/conda-cheatsheet.pdf

Optionally, you can also install CUDA if you have a CUDA-enabled GPU. 

In [ ]:
# conda create --name dlcourse python=3.7
# conda activate dlcourse
# conda install jupyter numpy matplotlib
# conda install -c anaconda cudatoolkit
# conda install pytorch torchvision -c pytorch

## PyTorch

WHAT IS PYTORCH?

It’s a Python-based scientific computing package targeted at two sets of audiences:

* A replacement for NumPy to use the power of GPUs
* a deep learning research platform that provides maximum flexibility and speed

### Tensors

Tensors are similar to NumPy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

PyTorch provides functions similar to numpy to create tensors:

In [ ]:
import torch

In [ ]:
x = torch.empty(2, 3) # Construct a 5x3 matrix, uninitialized
print(x)
print(x.size()) # torch.Size is a tuple, so it supports all tuple operations

tensor([[ 0.0000e+00, -2.5244e-29, -8.8390e+22],
        [ 2.5250e-29, -8.7954e+22, -2.0005e+00]])
torch.Size([2, 3])


In [ ]:
x = torch.rand(2, 3) # Construct a randomly initialized matrix
print(x)

tensor([[0.5942, 0.1682, 0.1602],
        [0.2713, 0.7422, 0.2591]])


In [ ]:
x = torch.zeros(5, 3, dtype=torch.long) # Construct a matrix filled zeros and of dtype long
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [ ]:
x = torch.tensor([5.5, 3]) # Construct a tensor directly from data
print(x)

tensor([5.5000, 3.0000])


In [ ]:
# create a tensor based on an existing tensor. These methods will reuse properties of the input tensor, e.g. dtype, unless new values are provided by user
x = x.new_ones(2, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.3531, -0.4168, -0.6011],
        [ 1.8560,  0.8437,  0.3677]])


### Operations

There are multiple syntaxes for operations. In the following example, we will take a look at the addition operation.

In [ ]:
y = torch.rand(2, 3)
print(x + y)

print(torch.add(x, y))

result = torch.empty(2, 3)
torch.add(x, y, out=result) # Providing an output tensor as argument
print(result)

y.add_(x) # In-place addition
print(y)

tensor([[ 1.1638, -0.0246, -0.5701],
        [ 2.7208,  1.3065,  0.8159]])
tensor([[ 1.1638, -0.0246, -0.5701],
        [ 2.7208,  1.3065,  0.8159]])
tensor([[ 1.1638, -0.0246, -0.5701],
        [ 2.7208,  1.3065,  0.8159]])
tensor([[ 1.1638, -0.0246, -0.5701],
        [ 2.7208,  1.3065,  0.8159]])


Any operation that mutates a tensor in-place is post-fixed with an \_. For example: x.copy_(y), x.t_(), will change x.

### Indexing

You can use standard numpy-like indexing

If you have a one element tensor, use .item() to get the value as a Python number

In [ ]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([0.8149])
0.8149430751800537


**Read later:**


  100+ Tensor operations, including transposing, indexing, slicing,
  mathematical operations, linear algebra, random numbers, etc.,
  are described here <https://pytorch.org/docs/torch>.

### NumPy Bridge

Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory
locations, and changing one will change the other.


In [ ]:
a = torch.ones(5)
print(a)
b = a.numpy() # Converts torch tensor to numpy array
print(b)

a.add_(1)
print(a)
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [ ]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a) # Converts numpy array to torch tensor
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


### CUDA Tensor
Tensors can be moved onto any device using the .to method.

In [ ]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

### Linear Regression example

torch.nn is a Neural Network package useful for constructing models which provides layers, activation functions, loss functions, etc. ( https://github.com/torch/nn )

In [ ]:
from itertools import count

import torch
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F

Define a polynomial function

In [ ]:
POLY_DEGREE = 4
W_target = torch.randn(POLY_DEGREE, 1) * 5
b_target = torch.randn(1) * 5

def f(x):
    """Approximated function."""
    return x.mm(W_target) + b_target.item()

Data loader and other utils

In [ ]:
def make_features(x):
    """Builds features i.e. a matrix with columns [x, x^2, x^3, x^4]."""
    x = x.unsqueeze(1)
    return torch.cat([x ** i for i in range(1, POLY_DEGREE+1)], 1)

def get_batch(batch_size=32):
    """Builds a batch i.e. (x, f(x)) pair."""
    random = torch.randn(batch_size)
    x = make_features(random)
    y = f(x)
    return x, y

def poly_desc(W, b):
    """Creates a string description of a polynomial."""
    result = 'y = '
    for i, w in enumerate(W):
        result += '{:+.2f} x^{} '.format(w, len(W) - i)
    result += '{:+.2f}'.format(b[0])
    return result

Defining the model

In [ ]:
# Define model
model = nn.Linear(W_target.size(0), 1)

Training loop

In [ ]:
for batch_idx in count(1):
    # Get data
    batch_x, batch_y = get_batch()

    # Reset gradients
    model.zero_grad()

    # Forward pass
    output = F.smooth_l1_loss(model(batch_x), batch_y)
    loss = output.item()

    # Backward pass
    output.backward()
    
    # Apply gradients
    for param in model.parameters():
        param.data.add_(-0.1 * param.grad.data)

    # Stop criterion
    if loss < 1e-3:
        break

print('Loss: {:.6f} after {} batches'.format(loss, batch_idx))
print('==> Learned function:\t' + poly_desc(model.weight.view(-1), model.bias))
print('==> Actual function:\t' + poly_desc(W_target.view(-1), b_target))

Loss: 0.000201 after 268 batches
==> Learned function:	y = +5.18 x^4 -2.41 x^3 -3.13 x^2 +1.85 x^1 -5.46
==> Actual function:	y = +5.19 x^4 -2.43 x^3 -3.12 x^2 +1.86 x^1 -5.46


Here's a slightly more advanced example of a Convolutional Neural Network for MNIST digit classification. <https://github.com/pytorch/examples/blob/master/mnist/main.py>

### Other Resources

Here are some other useful resources on PyTorch

* https://cs230-stanford.github.io/pytorch-getting-started.html
* https://github.com/jcjohnson/pytorch-examples
* https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html